In [37]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGPooling
from torch_geometric.nn import TopKPooling

from torch.optim.lr_scheduler import LambdaLR

from sklearn.mixture import GaussianMixture 
import json

import math
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['date'] = pd.to_datetime(data.date)
    print('Days: ',len(set(data.date)))
    return data

In [39]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table

In [40]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = '20190402_TaipeiEdgesDatewise.csv'
events_data =dataDir+'TaipeiEvents.csv'

In [41]:
file = dataDir + dataFile
dataRaw = loadData(file)

Raw shape:  (7374816, 5)
Days:  638


In [42]:
dataIn =  pd.pivot_table(dataRaw, values='amount', index=['date'],
                    columns=['end_id'], aggfunc=np.sum, fill_value=0)
dataIn.head(1)

end_id,BL01,BL02,BL03,BL04,BL05,BL06,BL07,BL08,BL09,BL10,...,R20,R21,R22,R22A,R23,R24,R25,R26,R27,R28
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,6326,11444,5696,16834,12256,25559,45609,27781,18745,33240,...,5597,5937,15733,12194,2972,2067,9518,8439,6473,73538


In [43]:
dataOut =  pd.pivot_table(dataRaw, values='amount', index=['date'],
                    columns=['start_id'], aggfunc=np.sum, fill_value=0)
dataOut.head(1)

start_id,BL01,BL02,BL03,BL04,BL05,BL06,BL07,BL08,BL09,BL10,...,R20,R21,R22,R22A,R23,R24,R25,R26,R27,R28
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,6292,10268,5591,15834,12955,26456,42135,29606,18572,34387,...,5734,6801,14919,12373,2943,1957,8887,8603,6870,76751


In [44]:
matrix = np.stack((dataOut.values, dataIn.values),-1)
matrix = matrix.astype(float)
matrix.shape

(638, 108, 2)

In [45]:
# for i in range(matrix.shape[1]):
#     for j in range(matrix.shape[2]):
#         matrix[:, i,j] = (matrix[:, i,j] - np.mean(matrix[:, i,j])) / (np.std(matrix[:, i,j]))

In [46]:
for i in range(matrix.shape[0]):
    for j in range(matrix.shape[2]):
        matrix[i,:,j] = (matrix[i,:,j] - np.mean(matrix[i,:,j])) / (np.std(matrix[i,:,j]))

In [47]:
with open('topology.json') as json_file:
    topology = json.load(json_file)

In [48]:
stationToIdx = dict(zip(dataOut.columns.values,range(len(dataOut.columns.values))))

In [49]:
edge_index = []
for t in topology:
    edge_index.append([stationToIdx[t[0]], stationToIdx[t[1]]])

In [50]:
stations = list(set(dataRaw.start_id))
n= len(stations)
#edge_index = [[a//n,a%n] for a in range(n*n)]
edge_index = torch.tensor(edge_index, dtype=torch.long)

In [51]:
dates = list(dataOut.index)
DOW = list(pd.to_datetime(dataOut.index.values).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [52]:
dataList = []

for i in range(len(DOW)):
    x = torch.tensor(matrix[i], dtype=torch.float)
    y = torch.tensor(np.array([DOW[i]]), dtype=torch.long)
    data = Data(x=x, edge_index=edge_index.t().contiguous(),y=y)
    dataList.append(data)

dataList[0]

Data(edge_index=[2, 232], x=[108, 2], y=[1])

In [53]:
n_classes = 2
featureDim = 2
hiddenDim = [20,1,20]
poolingRatio= 0.4
viewDim = math.ceil(0.4 * n)

In [54]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(featureDim, hiddenDim[0])
        self.conv2 = GCNConv(hiddenDim[0], hiddenDim[1])
        self.pool1 = SAGPooling(hiddenDim[1], ratio=poolingRatio, gnn='GCN')
        #self.pool1 = TopKPooling(hiddenDim[1], ratio=poolingRatio)
        self.linear1 = nn.Linear(viewDim, hiddenDim[2])
        self.linear2 = nn.Linear(hiddenDim[2], n_classes)

        self.init_weights(self.linear1)
        self.init_weights(self.linear2)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #print(x.shape)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #print(x.shape)
        z = self.pool1(x, edge_index)
        x, edge_index = z[0], z[1]
        #print(x.shape)
        x = x.view(-1,viewDim)
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        return F.log_softmax(x, dim=1)
        
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def nodeRepresentation(self, x):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        z = self.pool1(x, edge_index)
        x, edge_index = z[0], z[1]
        return x
    

In [55]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
#lambda1 = lambda epoch: 0.95 ** epoch
#scheduler = LambdaLR(optimizer, lr_lambda=lambda1)
criterion = nn.CrossEntropyLoss()

In [56]:
device

device(type='cuda')

In [57]:
#optimizer = torch.optim.Adam(model.parameters(), lr=0.000001, weight_decay=5e-4)
num_epochs = 100

In [58]:
model.train()

for epoch in range(num_epochs):
    epochLoss = 0
    for data in dataList:
        #data.view(1,-1,-1)
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
#         print(out.shape)
#         print(data.y.shape)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        
        epochLoss += loss.item()
    
#     if epoch == 500:
#         optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=5e-4)
    #scheduler.step()
    print('epoch [{}/{}], loss:{:.4f}'
    .format(epoch + 1, num_epochs, epochLoss))

epoch [1/100], loss:435.4681
epoch [2/100], loss:420.7648
epoch [3/100], loss:413.0564
epoch [4/100], loss:403.8638
epoch [5/100], loss:399.2929
epoch [6/100], loss:394.4417
epoch [7/100], loss:391.5916
epoch [8/100], loss:390.6196
epoch [9/100], loss:386.2043
epoch [10/100], loss:388.6095
epoch [11/100], loss:383.7925
epoch [12/100], loss:386.5532
epoch [13/100], loss:382.8591
epoch [14/100], loss:382.9174
epoch [15/100], loss:377.9371
epoch [16/100], loss:379.3211
epoch [17/100], loss:382.4608
epoch [18/100], loss:378.5219
epoch [19/100], loss:381.0330
epoch [20/100], loss:381.9929
epoch [21/100], loss:374.6030
epoch [22/100], loss:378.6011
epoch [23/100], loss:375.5596
epoch [24/100], loss:371.3869
epoch [25/100], loss:375.7138
epoch [26/100], loss:367.4384
epoch [27/100], loss:368.4948
epoch [28/100], loss:373.6587
epoch [29/100], loss:370.2087
epoch [30/100], loss:367.0232
epoch [31/100], loss:364.2914
epoch [32/100], loss:365.4538
epoch [33/100], loss:361.7958
epoch [34/100], los

In [59]:
model.eval()
with torch.no_grad():
    y = DOW
    predList = []
    for data in dataList:
        data = data.to(device)
        pred = model(data).cpu().numpy()
        predList.append(np.argmax(pred))
    
    print('Accuracy: {:.4f}'.format( np.sum(np.array(predList) == np.array(y)) / len(predList) ))

Accuracy: 0.7085


In [60]:
representation = np.empty((matrix.shape[0],viewDim))
model.eval()
with torch.no_grad():
    for i,data in enumerate(dataList):
        data = data.to(device)
        x = model.nodeRepresentation(data).cpu().numpy()
        representation[i] = x.reshape(viewDim)
        predList.append(np.argmax(pred))

In [61]:
representation.shape

(638, 44)

In [62]:
# import events data
events_data =dataDir+'TaipeiEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [63]:
df_events.head()

,Type,Name,Date
0,National holiday,Republic Day/New Year's Day observed,2017-01-02
1,National holiday,Chinese New Year's Eve,2017-01-27
2,National holiday,Chinese New Year's Day,2017-01-28
3,National holiday,Chinese New Year Holiday 1,2017-01-29
4,National holiday,Chinese New Year Holiday 2,2017-01-30


In [64]:
holidayDates = df_events[df_events.Type == 'National holiday'].Date

In [65]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = [str(d.date()) for d in dates]

In [66]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [67]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if(sum(rind)==0):
            print('All anomalies in {} iterations'.format(i+1))
            break
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [68]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [69]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [70]:
PCARec = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1


/home/urwa/miniconda3/envs/gcn/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.6
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.7
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5

In [71]:
PCARec

,Cat,F1,FPR,Precision,TPR,th
0,Global,NaN,0.009852,0.000000,0.000000,0.01
1,Global,0.041667,0.029557,0.052632,0.034483,0.03
2,Global,0.037037,0.039409,0.040000,0.034483,0.04
3,Global,0.033333,0.049261,0.032258,0.034483,0.05
4,Global,0.059701,0.059113,0.052632,0.068966,0.06
5,Global,0.054795,0.068966,0.045455,0.068966,0.07
6,Global,0.050000,0.080460,0.039216,0.068966,0.08
7,Global,0.043478,0.100164,0.031746,0.068966,0.10
8,Global,0.051282,0.201970,0.031496,0.137931,0.20
9,Global,0.054545,0.303777,0.031414,0.206897,0.30
